In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Vitamin_D_Levels"
cohort = "GSE35925"

# Input paths
in_trait_dir = "../../input/GEO/Vitamin_D_Levels"
in_cohort_dir = "../../input/GEO/Vitamin_D_Levels/GSE35925"

# Output paths
out_data_file = "../../output/preprocess/Vitamin_D_Levels/GSE35925.csv"
out_gene_data_file = "../../output/preprocess/Vitamin_D_Levels/gene_data/GSE35925.csv"
out_clinical_data_file = "../../output/preprocess/Vitamin_D_Levels/clinical_data/GSE35925.csv"
json_path = "../../output/preprocess/Vitamin_D_Levels/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE35925_family.soft.gz', 'GSE35925_series_matrix.txt.gz']
Identified SOFT files: ['GSE35925_family.soft.gz']
Identified matrix files: ['GSE35925_series_matrix.txt.gz']

Background Information:
!Series_title	"Calcitriol supplementation effects on Ki67 expression and transcriptional profile of breast cancer specimens from post-menopausal patients"
!Series_summary	"Background: Breast cancer patients present lower 1,25(OH)2D3 or 25(OH)D3 serum levels than unaffected women. Although 1,25(OH)2D3 pharmacological concentrations of 1,25(OH)2D3 may exert antiproliferative effects in breast cancer cell lines, much uncertainty remains about the effects of calcitriol supplementation in tumor specimens in vivo. We have evaluated tumor dimension (ultrassonography), proliferative index (Ki67 expression), 25(OH)D3 serum concentration and gene expression profile, before and after a short term calcitriol supplementation (dose to prevent osteoporosis) to post-menopausal p

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and summary, this is a study of breast cancer specimens with gene
# expression profiling using microarrays (U133 Plus 2.0 GeneChip), not just miRNA or methylation
is_gene_available = True

# 2.1 Data Availability
# For trait (Vitamin D Levels):
# From the summary, we know this study examined 25(OH)D3 serum concentration,
# but the data is not explicitly available in the sample characteristics dictionary
trait_row = None  # Vitamin D level data is not available in sample characteristics

# For age:
# Age is available in row 1 of the sample characteristics
age_row = 1

# For gender:
# Gender is available in row 0 of the sample characteristics, and all patients are female
gender_row = 0

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    # Since trait data isn't available, this function won't be used
    # But defining it for completeness
    if not value or ':' not in value:
        return None
    val = value.split(':', 1)[1].strip()
    try:
        return float(val)
    except:
        return None

def convert_age(value):
    # Convert age to continuous value
    if not value or ':' not in value:
        return None
    val = value.split(':', 1)[1].strip()
    try:
        return int(val)
    except:
        return None

def convert_gender(value):
    # Convert gender to binary (0 for female, 1 for male)
    if not value or ':' not in value:
        return None
    val = value.split(':', 1)[1].strip().lower()
    if 'female' in val:
        return 0
    elif 'male' in val:
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is not available in the sample characteristics
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we should skip this substep
# No need to call geo_select_clinical_features or save clinical data


False

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')

Gene expression data shape: (54675, 30)


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers shown in previous step
# These identifiers (e.g., '1007_s_at', '1053_at') are Affymetrix probe IDs from a microarray
# They are not human gene symbols and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Sample of gene expression data (first 5 rows, first 5 columns):
           GSM877494  GSM877495  GSM877496  GSM877497  GSM877498
ID                                                              
1007_s_at  11.333510  10.134981  11.504773  11.124785  11.144094
1053_at     8.367081   6.781699   7.152553   6.712648   6.979207
117_at      7.119038   6.212178   7.274306   6.750108   7.428198
121_at      8.201648   7.997442   8.637606   8.335036   8.053557
1255_g_at   3.864814   3.267283   3.474927   3.288460   2.955061



Platform information:
!Series_title = Calcitriol supplementation effects on Ki67 expression and transcriptional profile of breast cancer specimens from post-menopausal patients
!Platform_title = [HG-U133_Plus_2] Affymetrix Human Genome U133 Plus 2.0 Array
!Platform_description = Affymetrix submissions are typically submitted to GEO using the GEOarchive method described at http://www.ncbi.nlm.nih.gov/projects/geo/info/geo_affy.html
!Platform_description =
!Platform_description = June 03, 2009: annotation table updated with netaffx build 28
!Platform_description = June 06, 2012: annotation table updated with netaffx build 32
!Platform_description = June 23, 2016: annotation table updated with netaffx build 35
#Target Description =
#RefSeq Transcript ID = References to multiple sequences in RefSeq. The field contains the ID and Description for each entry, and there can be multiple entries per ProbeSet.
#Gene Ontology Biological Process = Gene Ontology Consortium Biological Process derive


Gene annotation columns:
['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']

Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'A

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the correct columns for mapping
# From the previous inspection, we can see that:
# - Gene identifiers in gene_data are probe IDs like '1007_s_at', which match the 'ID' column in gene_annotation
# - Gene symbols are stored in the 'Gene Symbol' column of gene_annotation

# 2. Extract gene mapping dataframe with the probe ID and gene symbol columns
prob_col = 'ID'
gene_col = 'Gene Symbol'
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

print("Gene mapping dataframe:")
print(f"Shape: {mapping_df.shape}")
print(mapping_df.head())

# 3. Convert probe-level measurements to gene-level expression data
# This will divide probe values across multiple genes and sum all contributions for each gene
gene_data = apply_gene_mapping(gene_data, mapping_df)

print("\nGene expression dataframe after mapping:")
print(f"Shape: {gene_data.shape}")
print(gene_data.head())

# Check how many genes we have after mapping
print(f"\nNumber of genes after mapping: {len(gene_data)}")


Gene mapping dataframe:
Shape: (45782, 2)
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A



Gene expression dataframe after mapping:
Shape: (21278, 30)
          GSM877494  GSM877495  GSM877496  GSM877497  GSM877498  GSM877499  \
Gene                                                                         
A1BG       5.616316   6.773878   5.610153   6.248181   5.881286   6.651108   
A1BG-AS1   5.794817   5.586119   6.158518   5.386011   6.405554   6.576005   
A1CF       7.951007   8.912474   8.437105   8.502681   7.911599   7.672422   
A2M       15.064945  15.318153  15.979660  16.447088  16.520041  16.582738   
A2M-AS1    5.150456   5.886847   5.095151   5.191165   7.678134   7.326865   

          GSM877500  GSM877501  GSM877502  GSM877503  ...  GSM877514  \
Gene                                                  ...              
A1BG       5.631434   5.051273   5.880273   6.100511  ...   5.424866   
A1BG-AS1   6.704271   6.987863   5.783158   6.023409  ...   5.434135   
A1CF       8.708905   7.364438   8.764073   8.270111  ...   8.128462   
A2M       16.101992  15.756496  

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# Since trait_row is None (as determined in Step 2), we know that trait data is not available
# We should record this in the metadata without final validation

# Create a placeholder dataframe with samples as rows for metadata needs
sample_df = pd.DataFrame(
    {trait: [None] * len(normalized_gene_data.columns)}, 
    index=normalized_gene_data.columns
)

# Save metadata recording that gene data is available but trait data is not
is_usable = validate_and_save_cohort_info(
    is_final=False,  # Set to False since we can't do final validation without trait data
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=False
)

# Since trait data is not available, we can't create usable linked data
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
print("Cannot save linked data due to missing trait information.")

Gene data shape after normalization: (19845, 30)
First few gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Normalized gene data saved to ../../output/preprocess/Vitamin_D_Levels/gene_data/GSE35925.csv
Data quality check result: Not usable
Cannot save linked data due to missing trait information.
